In [10]:
import requests
import pandas as pd
import time
import keys

In [11]:
headers = {
    "X-Auth-Token": keys.FOOTBALL_DATA_API_TOKEN
}

In [16]:
def get_matches_table(competition_id, matchday):
        url = f'https://api.football-data.org/v4/competitions/{competition_id}/matches?matchday={matchday}&season=2023'
        response = requests.get(url, headers=headers)
        data = response.json()
        #wait 10 seconds so all data from the api can be collected
        time.sleep(1)

        rows = []  # collect rows here

        for match in  data['matches']:
            hometeam = match['homeTeam']
            awayteam = match['awayTeam']
            matchscore = match['score']['fullTime']
            referee = match['referees'][0]
            row = {
                'match_id' : match['id'],
                'match_date' : match['utcDate'],
                'match_status' : match['status'],
                'matchday' : match['matchday'],
                'stage' : match['stage'],
                'hometeam_id' :hometeam['id'],
                'hometeam' : hometeam['name'],
                'awayteam_id' : awayteam['id'],
                'awayteam' : awayteam['name'],
                'hometeam_score' : matchscore['home'],
                'awayteam_score' : matchscore['away'],
                'referee' : referee['name']
            }
            rows.append(row)
        return pd.DataFrame(rows)


In [17]:
def merge_matchdays(comp_id):
    dataframes = []
    matchdays = list(range(1,39))
    for match in matchdays:
        df = get_matches_table(comp_id, match)
        time.sleep(5)
        dataframes.append(df) #collect all matchdays
    merged_df = pd.concat(dataframes, ignore_index=True)
    return merged_df

In [18]:
matchdays = merge_matchdays(2021)

In [19]:
matchdays.shape

(380, 12)

In [20]:
matchdays.head(10)

,match_id,match_date,match_status,matchday,stage,hometeam_id,hometeam,awayteam_id,awayteam,hometeam_score,awayteam_score,referee
0,435943,2023-08-11T19:00:00Z,FINISHED,1,REGULAR_SEASON,328,Burnley FC,65,Manchester City FC,0,3,Craig Pawson
1,435944,2023-08-12T12:00:00Z,FINISHED,1,REGULAR_SEASON,57,Arsenal FC,351,Nottingham Forest FC,2,1,Michael Oliver
2,435945,2023-08-12T14:00:00Z,FINISHED,1,REGULAR_SEASON,1044,AFC Bournemouth,563,West Ham United FC,1,1,Peter Bankes
3,435946,2023-08-12T14:00:00Z,FINISHED,1,REGULAR_SEASON,397,Brighton & Hove Albion FC,389,Luton Town FC,4,1,David Coote
4,435947,2023-08-12T14:00:00Z,FINISHED,1,REGULAR_SEASON,62,Everton FC,63,Fulham FC,0,1,Stuart Attwell
5,435948,2023-08-12T14:00:00Z,FINISHED,1,REGULAR_SEASON,356,Sheffield United FC,354,Crystal Palace FC,0,1,John Brooks
6,435949,2023-08-12T16:30:00Z,FINISHED,1,REGULAR_SEASON,67,Newcastle United FC,58,Aston Villa FC,5,1,Andy Madley
7,435950,2023-08-13T13:00:00Z,FINISHED,1,REGULAR_SEASON,402,Brentford FC,73,Tottenham Hotspur FC,2,2,Robert Jones
8,435951,2023-08-13T15:30:00Z,FINISHED,1,REGULAR_SEASON,61,Chelsea FC,64,Liverpool FC,1,1,Anthony Taylor
9,435952,2023-08-14T19:00:00Z,FINISHED,1,REGULAR_SEASON,66,Manchester United FC,76,Wolverhampton Wanderers FC,1,0,Simon Hooper


In [21]:
#Add time stamp to dataset name
from datetime import datetime

def save_dataframe(df, prefix, folder="../data"):
   
    filename = f"{folder}/{prefix}.csv"
    df.to_csv(filename, index=False)
    print(f"Saved: {filename}")

In [22]:
save_dataframe(matchdays, 'matches')

Saved: ../data/matches.csv


In [125]:
def get_teamdata():
    url = 'https://api.football-data.org/v4//competitions/2021/teams?season=2023'
    response = requests.get(url, headers=headers)
    data = response.json()
    time.sleep(1)

    rows = []
    for team in data['teams']:
        row ={
            'team_id' : team['id'],
            'team_name' : team['name'],
            'tla' : team['tla'],
            'year_founded' : team['founded'],
            'clubColors' : team['clubColors'],
            'stadium' : team['venue'],
            'crest' : team['crest'],
            }
        rows.append(row)

    return pd.DataFrame(rows)


In [126]:
team_df = get_teamdata()

In [127]:
team_df

,team_id,team_name,tla,year_founded,clubColors,stadium,crest
0,57,Arsenal FC,ARS,1886.0,Red / White,Emirates Stadium,https://crests.football-data.org/57.png
1,58,Aston Villa FC,AVL,1872.0,Claret / Sky Blue,Villa Park,https://crests.football-data.org/58.png
2,61,Chelsea FC,CHE,1905.0,Royal Blue / White,Stamford Bridge,https://crests.football-data.org/61.png
3,62,Everton FC,EVE,1878.0,Blue / White,Goodison Park,https://crests.football-data.org/62.png
4,63,Fulham FC,FUL,1879.0,White / Black,Craven Cottage,https://crests.football-data.org/63.png
5,64,Liverpool FC,LIV,1892.0,Red / White,Anfield,https://crests.football-data.org/64.png
6,65,Manchester City FC,MCI,1880.0,Sky Blue / White,Etihad Stadium,https://crests.football-data.org/65.png
7,66,Manchester United FC,MUN,1878.0,Red / White,Old Trafford,https://crests.football-data.org/66.png
8,67,Newcastle United FC,NEW,1881.0,Black / White,St. James' Park,https://crests.football-data.org/67.png
9,73,Tottenham Hotspur FC,TOT,1882.0,Navy Blue / White,Tottenham Hotspur Stadium,https://crests.football-data.org/73.png


In [202]:
save_dataframe(team_df, 'teams_2023_24(raw)')

Saved: ../data/teams_2023_24(raw)_20250720_221521.csv


In [128]:
def get_standings():
    url = 'https://api.football-data.org/v4//competitions/2021/standings?season=2023'
    response = requests.get(url, headers=headers)
    data = response.json()
    time.sleep(1)
    
    rows = []
    
    for team in data['standings'][0]['table']:
        row = {
            'team_id' : team['team']['id'],
            'team_name' : team['team']['name'],
            'position' : team['position'],
            'games_played' : team['playedGames'],
            'form': team['form'],
            'wins': team['won'],
            'draws': team['draw'],
            'losses': team['lost'],
            'points': team['points'],
            'goalsFor': team['goalsFor'],
            'goals_against': team['goalsAgainst'],
            'goal_difference': team['goalDifference']
        }

        rows.append(row)
        
    return pd.DataFrame(rows)

In [129]:
standings = get_standings() 

In [130]:
standings

,team_id,team_name,position,games_played,form,wins,draws,losses,points,goalsFor,goals_against,goal_difference
0,65,Manchester City FC,1,38,"W,W,W,W,W",28,7,3,91,96,34,62
1,57,Arsenal FC,2,38,"W,W,W,W,W",28,5,5,89,91,29,62
2,64,Liverpool FC,3,38,"W,D,W,D,L",24,10,4,82,86,41,45
3,58,Aston Villa FC,4,38,"L,D,L,D,W",20,8,10,68,76,61,15
4,73,Tottenham Hotspur FC,5,38,"W,L,W,L,L",20,6,12,66,74,61,13
5,61,Chelsea FC,6,38,"W,W,W,W,W",18,9,11,63,77,63,14
6,67,Newcastle United FC,7,38,"W,L,D,W,W",18,6,14,60,85,62,23
7,66,Manchester United FC,8,38,"W,W,L,L,D",18,6,14,60,57,58,-1
8,563,West Ham United FC,9,38,"L,W,L,D,L",14,10,14,52,60,74,-14
9,354,Crystal Palace FC,10,38,"W,W,W,D,W",13,10,15,49,57,58,-1


In [131]:
team_stats = standings.merge(team_df[['team_id','tla']], on='team_id')

In [132]:
team_stats

,team_id,team_name,position,games_played,form,wins,draws,losses,points,goalsFor,goals_against,goal_difference,tla
0,65,Manchester City FC,1,38,"W,W,W,W,W",28,7,3,91,96,34,62,MCI
1,57,Arsenal FC,2,38,"W,W,W,W,W",28,5,5,89,91,29,62,ARS
2,64,Liverpool FC,3,38,"W,D,W,D,L",24,10,4,82,86,41,45,LIV
3,58,Aston Villa FC,4,38,"L,D,L,D,W",20,8,10,68,76,61,15,AVL
4,73,Tottenham Hotspur FC,5,38,"W,L,W,L,L",20,6,12,66,74,61,13,TOT
5,61,Chelsea FC,6,38,"W,W,W,W,W",18,9,11,63,77,63,14,CHE
6,67,Newcastle United FC,7,38,"W,L,D,W,W",18,6,14,60,85,62,23,NEW
7,66,Manchester United FC,8,38,"W,W,L,L,D",18,6,14,60,57,58,-1,MUN
8,563,West Ham United FC,9,38,"L,W,L,D,L",14,10,14,52,60,74,-14,WHU
9,354,Crystal Palace FC,10,38,"W,W,W,D,W",13,10,15,49,57,58,-1,CRY


In [133]:
save_dataframe(team_stats, 'team_stats(raw)')

Saved: ../data/team_stats(raw)_20250723_013940.csv


In [26]:
def get_table(id, page):
    headers = {'x-apisports-key': keys.API_SPORTS_KEY}
    url = f'https://v3.football.api-sports.io/players?league=39&season=2023&team={id}&page={page}'
    
    response = requests.get(url, headers=headers)
    data = response.json()
    #wait 10 seconds so all data from the api can be collected
    time.sleep(1)
    
    rows = []  # Collect rows here
    
    for player in data['response']:
        stats = player['statistics'][0]
        if stats['games']['appearences'] is None:
            continue
        
        row = {
            'player_name': player['player']['name'],
            'age': player['player']['age'],
            'nationality': player['player']['nationality'],
            'club': stats['team']['name'],
            'position': stats['games']['position'],
            'appearances': stats['games']['appearences'],
            'starts': stats['games']['lineups'],
            'minutes': stats['games']['minutes'],
            'goals': stats['goals']['total'],
            'assists': stats['goals']['assists'],
            'total_passes': stats['passes']['total'],
            'penalty_goals': stats['penalty']['scored'],
            'yellow_card': stats['cards']['yellow'],
            'red_card': stats['cards']['red']
        }
        rows.append(row)
    
    return pd.DataFrame(rows)


In [27]:
def merge_df(team_id):
    dataframes = []
    for page in [1, 2, 3]:
        df = get_table(team_id, page)
        time.sleep(5)
        dataframes.append(df)  # collect the returned dataframe
    merged_df = pd.concat(dataframes, ignore_index=True)
    return merged_df

In [28]:
#merge all teams
def merge_all_teams(team_ids):
    all_data = []
    for team_id in team_ids:
        team_df = merge_df(team_id)
        time.sleep(5)
        all_data.append(team_df)
    return pd.concat(all_data, ignore_index=True)



In [29]:
# Get all the players data and Merge all data:
#Id for all clubs in the league
clubs = [50, 40, 42, 33, 47, 48 , 39, 45, 55, 49, 52, 36, 34, 51, 35, 44, 1359, 62, 66, 65]
scorer_df = merge_all_teams(clubs)

C:\Users\okonk\AppData\Local\Temp\ipykernel_14432\1721752609.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat(dataframes, ignore_index=True)


In [31]:
scorer_df.shape

(894, 14)

In [34]:
save_dataframe(scorer_df, 'players')

Saved: ../data/players_20250722_174833.csv


# Data Cleaning

In [206]:
df = pd.read_csv('../data/matches_2023_24(raw)_20250720_221550.csv')

In [208]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   match_id        380 non-null    int64 
 1   match_date      380 non-null    object
 2   match_status    380 non-null    object
 3   matchday        380 non-null    int64 
 4   stage           380 non-null    object
 5   hometeam_id     380 non-null    int64 
 6   hometeam        380 non-null    object
 7   awayteam_id     380 non-null    int64 
 8   awayteam        380 non-null    object
 9   hometeam_score  380 non-null    int64 
 10  awayteam_score  380 non-null    int64 
 11  referee         380 non-null    object
dtypes: int64(6), object(6)
memory usage: 35.8+ KB


In [209]:
df['match_date'] = pd.to_datetime(df['match_date'])

In [ ]:
df['date']

In [210]:
df.drop(columns=['hometeam_id', 'awayteam_id'], inplace =True)

In [211]:
df = df.rename(columns={
    'id': 'match_id',
    'match_date': 'date',
    'hometeam': 'home_team',
    'awayteam': 'away_team',
    'hometeam_score': 'home_goals',
    'awayteam_score': 'away_goals'
})

In [215]:
df.drop('referee', axis=1, inplace=True)

In [212]:
def get_result(row):
    if row['home_goals'] > row['away_goals']:
        return 'H'
    elif row['home_goals'] < row['away_goals']:
        return 'A'
    else:
        return 'D'

df['result'] = df.apply(get_result, axis=1)

In [213]:
df.head()

,match_id,date,match_status,matchday,stage,home_team,away_team,home_goals,away_goals,referee,result
0,435943,2023-08-11 19:00:00+00:00,FINISHED,1,REGULAR_SEASON,Burnley FC,Manchester City FC,0,3,Craig Pawson,A
1,435944,2023-08-12 12:00:00+00:00,FINISHED,1,REGULAR_SEASON,Arsenal FC,Nottingham Forest FC,2,1,Michael Oliver,H
2,435945,2023-08-12 14:00:00+00:00,FINISHED,1,REGULAR_SEASON,AFC Bournemouth,West Ham United FC,1,1,Peter Bankes,D
3,435946,2023-08-12 14:00:00+00:00,FINISHED,1,REGULAR_SEASON,Brighton & Hove Albion FC,Luton Town FC,4,1,David Coote,H
4,435947,2023-08-12 14:00:00+00:00,FINISHED,1,REGULAR_SEASON,Everton FC,Fulham FC,0,1,Stuart Attwell,A


In [220]:
save_dataframe(df, 'matches_clean')

Saved: ../data/matches_clean_20250720_223207.csv


In [104]:
player = pd.read_csv('../data/players_20250722_174833.csv')

In [105]:
player.head()

,player_name,age,nationality,club,position,appearances,starts,minutes,goals,assists,total_passes,penalty_goals,yellow_card,red_card
0,B. Knight,23,England,Manchester City,Midfielder,0,0,0,0,NaN,NaN,NaN,0,0
1,K. Walker,35,England,Manchester City,Defender,32,30,2767,0,4.0,2136.0,0.0,2,0
2,Max Alleyne,20,England,Manchester City,Defender,0,0,0,0,NaN,NaN,NaN,0,0
3,S. Carson,40,England,Manchester City,Goalkeeper,0,0,0,0,NaN,NaN,0.0,0,0
4,Mahamadou Susoho Sissoho,20,Spain,Manchester City,Midfielder,0,0,0,0,NaN,NaN,0.0,0,0


In [106]:
player.isna().sum()

player_name        0
age                0
nationality        0
club               0
position           0
appearances        0
starts             0
minutes            0
goals              0
assists          491
total_passes     319
penalty_goals    143
yellow_card        0
red_card           0
dtype: int64

In [107]:
player.fillna(0, inplace = True)

In [108]:
player = player[player['appearances'] >= 5].reset_index()

In [109]:
duplicates = player.duplicated(subset='player_name' ,keep=False)
player[duplicates]

,index,player_name,age,nationality,club,position,appearances,starts,minutes,goals,assists,total_passes,penalty_goals,yellow_card,red_card
66,126,Reguilón,29,Spain,Manchester United,Defender,9,4,410,0,4.0,412.0,0.0,3,0
86,169,A. Onana,29,Cameroon,Manchester United,Goalkeeper,38,38,3420,0,0.0,1373.0,0.0,5,0
93,190,B. Johnson,24,Wales,Tottenham,Attacker,32,23,2091,5,0.0,58.0,0.0,3,0
132,254,B. Johnson,25,England,West Ham,Defender,14,4,533,0,0.0,262.0,0.0,1,0
141,280,B. Traoré,24,Mali,Wolves,Midfielder,24,7,805,0,0.0,443.0,0.0,4,0
177,336,A. Onana,24,Belgium,Everton,Midfielder,30,23,2091,2,0.0,1023.0,0.0,5,0
179,344,Reguilón,29,Spain,Brentford,Defender,16,14,1122,0,4.0,412.0,0.0,4,1
209,414,A. Broja,24,Albania,Chelsea,Attacker,13,6,453,1,0.0,93.0,0.0,0,0
267,510,A. Broja,24,Albania,Fulham,Attacker,8,0,81,0,0.0,93.0,0.0,0,0
411,769,B. Traoré,23,Côte d'Ivoire,Sheffield Utd,Attacker,8,3,384,0,0.0,67.0,0.0,0,0


In [110]:
player.iloc[86, 1] = 'Andre Onana' 
player.iloc[177, 1] = 'Amadou Onana' 
player.iloc[141, 1] = 'Boubacar Traoré'
player.iloc[411, 1] = 'Bénie Traoré'
player.iloc[93, 1] = 'Brennan Johnson'
player.iloc[132, 1] = 'Ben Johnson'

In [111]:
duplicates = player.duplicated(subset='player_name' ,keep=False)
player.drop_duplicates(keep=False, inplace = True)

In [112]:
player.drop('index',axis=1, inplace=True)

In [113]:
player.head()

,player_name,age,nationality,club,position,appearances,starts,minutes,goals,assists,total_passes,penalty_goals,yellow_card,red_card
0,K. Walker,35,England,Manchester City,Defender,32,30,2767,0,4.0,2136.0,0.0,2,0
1,K. De Bruyne,34,Belgium,Manchester City,Midfielder,18,15,1228,4,10.0,804.0,0.0,2,0
2,Sergio Gómez,25,Spain,Manchester City,Midfielder,6,0,48,0,1.0,49.0,0.0,0,0
3,J. Álvarez,25,Argentina,Manchester City,Attacker,36,31,2658,11,8.0,1110.0,2.0,2,0
4,Rodri,29,Spain,Manchester City,Midfielder,34,34,2937,8,9.0,3630.0,0.0,8,1


In [114]:
player['club'].unique()

array(['Manchester City', 'Liverpool', 'Arsenal', 'Manchester United',
       'Tottenham', 'West Ham', 'Wolves', 'Everton', 'Brentford',
       'Chelsea', 'Crystal Palace', 'Fulham', 'Newcastle', 'Brighton',
       'Bournemouth', 'Burnley', 'Luton', 'Sheffield Utd', 'Aston Villa',
       'Nottingham Forest'], dtype=object)

In [115]:
team_id = {
            'Manchester City':'65',
            'Liverpool':'64',
            'Arsenal':'57',
            'Manchester United':'66',
            'Tottenham':'73',
            'West Ham':'563',
            'Wolves':'76',
            'Everton':'62',
            'Brentford':'402',
            'Chelsea':'61',
            'Crystal Palace':'354',
            'Fulham':'63',
            'Newcastle':'67',
            'Brighton':'397',
            'Bournemouth':'1044',
            'Burnley':'397',
            'Luton':'389',
            'Sheffield Utd':'356',
            'Aston Villa':'58',
            'Nottingham Forest':'351'
          }
player['team_id'] = player['club'].map(team_id)

In [116]:
player.rename(columns={
    'club':'team',
    'player_name':'name',
}, inplace=True)

In [117]:
player.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478 entries, 0 to 477
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           478 non-null    object 
 1   age            478 non-null    int64  
 2   nationality    478 non-null    object 
 3   team           478 non-null    object 
 4   position       478 non-null    object 
 5   appearances    478 non-null    int64  
 6   starts         478 non-null    int64  
 7   minutes        478 non-null    int64  
 8   goals          478 non-null    int64  
 9   assists        478 non-null    float64
 10  total_passes   478 non-null    float64
 11  penalty_goals  478 non-null    float64
 12  yellow_card    478 non-null    int64  
 13  red_card       478 non-null    int64  
 14  team_id        478 non-null    object 
dtypes: float64(3), int64(7), object(5)
memory usage: 56.1+ KB


In [118]:
player['assists'] = player['assists'].astype(int)
player['total_passes'] = player['total_passes'].astype(int)
player['penalty_goals'] = player['penalty_goals'].astype(int)

In [119]:
save_dataframe(player, 'players_cleaned')

Saved: ../data/players_cleaned_20250722_204616.csv
